# Deep Reinforcement Learning Udacity Nanodegree
# Project 3: Collaboration and Competition

## Environment description
The [Unity ML-Agents toolkit](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Learning-Environment-Examples.md) contains the [Tennis Environment](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Learning-Environment-Examples.md#tennis) in which two tennis rackets keep the ball in the game by bouncing it over the net. 
In the [Collaboration and Competition Project](https://github.com/udacity/deep-reinforcement-learning/tree/master/p3_collab-compet) of the [Deep Reinforcement Learning Udacity Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893), the Tennis environment has been slightly adapted by Udacity, so there might be differences to the original environment.

[//]: # (Image References)

[image1]: https://user-images.githubusercontent.com/10624937/42135623-e770e354-7d12-11e8-998d-29fc74429ca2.gif "Trained Agent"

![image1]

If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. 3 observations are stacked so that each agent receives a state of size 24. Each agent receives its own, local observation.  Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 


In [2]:
env = UnityEnvironment(file_name='Tennis_Linux/Tennis.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


## Approach: Deep Deterministic Policy Gradient method

Due to the continuous action space, classical value-based methods are hard to apply. Instead, we solve the environment using the Multiagent Deep Deterministic Policy Gradient (DDPG) method. This is a flavour of an actor-critic method, where the actor is trained to maximize the expected outcome as is predicted by the critic network. As a twist, our critics also see the observation space and next actions of the opponents. This is allowed, as this only happens during training. When we test the agents, we don't need the critics anymore, and the actors only rely on their local observations.


### Implementation
I worked on this project as part of the [Deep Reinforcement Learning Udacity Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) and turned the DDPG agent code from the [Pendulum project in Udacity's Deep Reinforcement Learning Repository](https://github.com/udacity/deep-reinforcement-learning/tree/master/ddpg-pendulum) into a Multiagent version. The most important changes are as follows.

#### Multiagent version
The original code only has a DDPG agent with actor and critic. In our environment, we have two agents each with their own actor and critic and own observations.

Computing the next actions is pretty straightforward, as the actor only needs to know about its own current state.

However, during learning, we also need to compute and improve the critic, which is more complicated as it not only relies on the  observations and actions of the current agent, but also the opponent's observations and actions. Please see the publication [Multi Agent Actor Critic for Mixed Cooperative Competitive environments](https://papers.nips.cc/paper/7217-multi-agent-actor-critic-for-mixed-cooperative-competitive-environments.pdf) by OpenAI for details.

From a technical perspective, this means that when computing the Q targets
```
Q_targets = r + γ * critic_target(next_state, actor_target(next_state))
```
we also need to incorporate the actor_target of the other actor. The same applies when we want to compute the current actor's loss, because then we also need the critic that takes the other actor's action into account.

In order to shield the Jupyter Notebook user from these details, I implemented the class `MultiagentWrapper` that offers `learn` and `act` methods that would take care of the details related to the Multiagent implementation.

#### Ornstein-Uhlenbeck noise generation

In the original code, the noise is generated using this code here

```
dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for i in range(len(x))])
```
However, `random.random()` generates uniformly distributed random numbers with mean 0.5. I changed this to standard normally distributed values using `np.random.randn()` as this is more customary for Ornstein-Uhlenbeck.

More importantly, I scaled the noise down by a factor of 0.7 that is multiplied by 0.9999 in every episode. This results in a gradually decreasing noise level and shift from exploration to exploitation the longer the training runs. When testing the agents the noise is set to 0.0 completely.

#### Experience buffer
In the [continuous control project](https://github.com/hullmann/continuouscontrol-DDPG) the 20 agents learned from each other by using a shared replay buffer. This sounds desirable also for the tennis environment, but there is no proper experience sharing between our two agents because I wasn't sure that all observations and actions are completely symmetrical. This means, their observations and and actions are stored separately in the same buffer.

However, what we do for coupling the two agents, is that we feed both critic networks also with the observations and actions of the other agent.

#### Deep Neural Network architecture

For both actor and critic, we use linear neural networks with two hidden layers with Relu activation functions. Their sizes are 265 and 128 nodes, respectively.

The input layer of the actor has 24 nodes, as this corresponds to the state size. The input layer of the critic has 52 nodes, as this corresponds to 2 * (state size + action size).

For the actor, the output layer has 2 nodes, which corresponds to the action size, and a tanh-activation function to map to the interval (-1, 1). The critic has naturally only one output node without activation function to allow for any numerical output value.

It is worth mentioning that a batch normalization layer is needed between both hidden layers. Leaving it out or putting it at another position resulted in a failure of the agents to solve the environment. I don't know why the batch normalization layer at this position is so important, but I assume it has to do with the high level of noise and the initially very unstable and erratic movements.


#### Hyperparameters
I chose the following hyperparameters:
- the discount factor gamma=0.99 is slightly lower than 1 because the episodes can run quite long
- the batch size is 512, because I read that larger batch sizes allow for higher learning rates
- the actor learning rate is 1e-3 and the critic is 1e-2. As the actor relies on the critic, having a higher learning rate for the critic makes sense. Generally, the learning rates seem quite high, but I found the results convincing
- every 4 time steps we train both agents with one batch from the replay buffer. This doesn't consume too much computing resources, but still results in a fast training (albeit varying a lot with the random seed)
- as discussed above, we scale the noise with a factor of 0.7 and gradually reduce the noise by a factor of 0.9999 in each episode 
- we update the target networks for actors and critis with a tau of 0.01, which again seems high but works well

Please note that the training does take some time, which limited the amount of parameters I could try. This means that there are probably much better choices out there, but I found this combination works quite well.

### Training results

With the preconfigured random seed of 7 it needs 1054 episodes, with a random seed of 3 it needs 2755 episodes. Training is quite challenging as initially the players miss the ball more often than not, and the reward plots looks quite poor and instable. So every modification of the neural takes quite long to confirm whether it actually results in a working solution.

Please see the reward plot for both agents and the running mean of their maximum value over the last 100 values for a random seed of 7:

[image2]: https://raw.githubusercontent.com/hullmann/multiagent-DDPG/master/scoregraph.png "Reward plot"

![image2]

The environment is solved as soon as the average exceeds a score of +0.5.

The [weights of the agents](https://github.com/hullmann/multiagent-DDPG/tree/master/model_weights) are part of the repository as well as a [video of the trained agents](https://raw.githubusercontent.com/hullmann/multiagent-DDPG/master/Trained_Agent_Recording.mp4) (the latter for random seed 3).

# Ideas for future work

This environment wasn't easy to solve. I'm interested in the following three topics
- why exactly is the batch normalization layer exactly between the two hidden layers so important?
- how can we regularize our networks (dropout layers, etc.) such that training runs more smoothly and the agents are less erratic in the end?
- would a prioritized replay buffer speed up training and/or make it more stable?